In [14]:
from scipy import integrate as sp
import numpy as np
import SALib
from SALib import sample
from SALib.analyze import sobol

# definition of the system of ODEs
def iron(XYZ,t,a12,a21,a23,a32,b13,b31,I):
  X1,X2,X3=XYZ
  dX1=-a12*(X1)+a21*(X2)-b13*(X1)+b31*(X3)
  dX2=-a23*(X2)-a21*(X2)+a12*(X1)+a32*(X3)
  dX3=-a32*(X3)-b31*(X3)+a23*(X2)+b13*(X1)-I
  return dX1,dX2,dX3;

# default parameter values
a12=0.0005 
a21=0.00001
a23=0.0003 
a32=0.0002 
b13=0.0001 
b31=0.000001 
I=0.001 

# initial condition
XYZ0=[1000.,30.,10.]
X10=1000.
X20=50.
X30=30.

# tmie steps
t=np.linspace(0,100,1000) #(start,stop,num samples to generate)

# example single calculation
XYZ=sp.odeint(iron,XYZ0,t,args=(a12,a21,a23,a32,b13,b31,I))

### Sobol analysis ###
# defining problem
# can add the 'I' parameter
# assumed that range for each parameter is 80-120% of value assumed above
# can be changed
problem = {
  'num_vars': 9, #a's, b's and initial condition
  'names': ['a12', 'a21','a23','a32','b13','b31','x1_0','x2_0','x3_0'],
  'bounds':  np.column_stack((
      np.array([a12, a21,a23,a32,b13,b31,XYZ0[0],XYZ0[1],XYZ0[2]])*0.8,
      np.array([a12, a21,a23,a32,b13,b31,XYZ0[0],XYZ0[1],XYZ0[2]])*1.2))
}

# Generate samples
vals = sample.sobol.sample(problem, 512)

# initializing matrix to store output
Y = np.zeros([len(vals),1])

# Run model (example)
# numerically soves the ODE
# output is X1, X2, and X3 at the end time step
# could save output for all time steps if desired, but requires more memory
Y = np.zeros([len(vals),3])
for i in range(len(vals)):
  Y[i][:] = sp.odeint(iron,[vals[i][6],vals[i][7],vals[i][8]],t,
    args=(vals[i][0],vals[i][1],vals[i][2],vals[i][3],vals[i][4],vals[i][5],I))[len(XYZ)-1]


# completing soboal analysis for each X1, X2, and X3
print('\n\n====X1 Sobol output====\n\n')
Si_X1 = sobol.analyze(problem, Y[:,0], print_to_console=True)
print('\n\n====X2 Sobol output====\n\n')
Si_X2 = sobol.analyze(problem, Y[:,1], print_to_console=True)
print('\n\n====X3 Sobol output====\n\n')
Si_X3 = sobol.analyze(problem, Y[:,2], print_to_console=True)



====X1 Sobol output====


                ST       ST_conf
a12   2.525843e-03  3.396108e-04
a21   3.120556e-09  3.926174e-10
a23   4.148668e-13  3.904911e-14
a32   1.613874e-14  1.771657e-15
b13   1.011013e-04  1.286422e-05
b31   2.544353e-12  2.893744e-13
x1_0  9.979849e-01  8.800250e-02
x2_0  9.406015e-10  9.985261e-11
x3_0  1.271842e-12  1.375707e-13
                S1       S1_conf
a12   2.527058e-03  5.920045e-03
a21  -4.967540e-06  6.786597e-06
a23  -1.736649e-09  7.307639e-08
a32  -1.092839e-10  1.489115e-08
b13  -3.735550e-04  1.408794e-03
b31   1.510633e-07  2.060379e-07
x1_0  9.975507e-01  1.014878e-01
x2_0  2.805713e-09  4.125983e-06
x3_0  1.065620e-09  1.535533e-07
                        S2       S2_conf
(a12, a21)   -1.177950e-04  8.694011e-03
(a12, a23)   -1.177984e-04  8.694112e-03
(a12, a32)   -1.177984e-04  8.694111e-03
(a12, b13)   -1.182038e-04  8.705271e-03
(a12, b31)   -1.177984e-04  8.694110e-03
(a12, x1_0)  -5.775566e-05  1.163198e-02
(a12, x2_0)  -1.178185e-0